In [369]:
# Manipulação e tratamento das bases
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)
from scipy import stats
#!pip install -U scikit-learn

#Visualização gráfica
%matplotlib inline
import seaborn as sns
from matplotlib import pyplot
import matplotlib.pyplot as plt
import plotly.express as px
import io
from sklearn import tree

#Pré-Processamento das bases
!pip install imblearn
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
#!pip install Boruta
from boruta import BorutaPy
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectFromModel

#Modelagem de Dados
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier, BaggingClassifier, AdaBoostClassifier, VotingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import RidgeClassifier, LogisticRegression, SGDClassifier, PassiveAggressiveClassifier, Perceptron
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.neural_network import MLPClassifier, BernoulliRBM
from sklearn.calibration import CalibratedClassifierCV
from sklearn.dummy import DummyClassifier
from sklearn.semi_supervised import LabelSpreading, LabelPropagation
#! pip install XGboost
from xgboost import XGBClassifier
#! pip install lightgbm
from lightgbm import LGBMClassifier
#! pip install catboost
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree, ExtraTreeClassifier
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve
from yellowbrick.classifier import ROCAUC
from yellowbrick.classifier import ConfusionMatrix

# Leitura da Base Antes da Vacina

In [370]:
Antes= pd.read_csv('Base_Final.csv', sep= ',')
Antes= Antes.loc[Antes['CO_MUN_NOT'].isin([500270])]
Antes=Antes[(Antes['Periodo']==1.0)]
Antes = Antes.drop(columns=["CO_MUN_NOT", "Periodo"])
Antes.head()

,IDADE_ANOS,CS_SEXO_M,CS_RACA_4.0,FEBRE_1.0,DISPNEIA_1.0,SATURACAO_1.0,UTI_1.0,SUPORT_VEN_1.0,SUPORT_VEN_2.0,PCR_RESUL_2.0,TOSSE_1.0,DESC_RESP_1.0,FATOR_RISC_2,EVOLUCAO
21,60.0,1,1,1,1,1,1,1,0,1,1,1,0,2.0
69,10.0,0,1,1,1,1,0,0,1,1,0,1,0,1.0
237,80.0,1,0,1,1,0,0,0,0,0,1,1,0,2.0
244,58.0,1,1,1,1,1,1,1,0,0,1,1,0,1.0
260,77.0,1,1,0,1,1,1,1,0,0,1,1,0,1.0


In [371]:
Antes= Antes.reset_index(drop=True)
Antes.head()

,IDADE_ANOS,CS_SEXO_M,CS_RACA_4.0,FEBRE_1.0,DISPNEIA_1.0,SATURACAO_1.0,UTI_1.0,SUPORT_VEN_1.0,SUPORT_VEN_2.0,PCR_RESUL_2.0,TOSSE_1.0,DESC_RESP_1.0,FATOR_RISC_2,EVOLUCAO
0,60.0,1,1,1,1,1,1,1,0,1,1,1,0,2.0
1,10.0,0,1,1,1,1,0,0,1,1,0,1,0,1.0
2,80.0,1,0,1,1,0,0,0,0,0,1,1,0,2.0
3,58.0,1,1,1,1,1,1,1,0,0,1,1,0,1.0
4,77.0,1,1,0,1,1,1,1,0,0,1,1,0,1.0


Verificação do Balanceamento

In [372]:
Antes["EVOLUCAO"].value_counts(normalize=True)

1.0    0.762315
2.0    0.237685
Name: EVOLUCAO, dtype: float64

Split da Base

In [373]:
X = Antes.drop(columns=['EVOLUCAO'])
y = Antes['EVOLUCAO']

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state=42)

In [374]:
Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape

((6493, 13), (2784, 13), (6493,), (2784,))

In [375]:
smote = SMOTE(sampling_strategy = 'minority', random_state = 42)
Xtrain_over, ytrain_over = smote.fit_resample(Xtrain,ytrain)

Xtest_over, ytest_over = smote.fit_resample(Xtest,ytest)
Xtrain_over.shape, ytrain_over.shape, Xtest_over.shape, ytest_over.shape

((9818, 13), (9818,), (4326, 13), (4326,))

Início da Modelagem

In [376]:
# Parâmetros
num_folds = 10
scoring = 'accuracy'

# Criação dos modelos
models = []
models.append(('LR', LogisticRegression()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('BGC', BaggingClassifier()))
models.append(('RDF', RandomForestClassifier()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('QDA', QuadraticDiscriminantAnalysis()))
models.append(('RDG', RidgeClassifier()))
models.append(('ADA', AdaBoostClassifier()))
models.append(('GRA', GradientBoostingClassifier()))
models.append(('CAT', CatBoostClassifier()))
models.append(('LGB', LGBMClassifier()))
models.append(('XGB', XGBClassifier()))
models.append(('NCE', NearestCentroid()))
models.append(('XTC', ExtraTreesClassifier()))
models.append(('SGD', SGDClassifier()))
models.append(('PAC', PassiveAggressiveClassifier()))
models.append(('RCV', RidgeClassifier()))
models.append(('LSV', LinearSVC()))
models.append(('PCP', Perceptron()))
models.append(('BNB', BernoulliNB()))
models.append(('DUC', DummyClassifier()))
models.append(('XTR', ExtraTreeClassifier()))

In [377]:
random_state=7 # definindo uma semente global
# Avaliação dos modelos
results = []
names = []
for name, model in models:
    kfold = KFold(n_splits=num_folds)
    cv_results = cross_val_score(model, Xtrain_over, ytrain_over, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

C:\Users\Windows10\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Windows10\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
 

LR: 0.794661 (0.016486)
KNN: 0.848152 (0.052643)
CART: 0.844073 (0.041440)
NB: 0.784170 (0.015304)
BGC: 0.866885 (0.032032)
RDF: 0.865869 (0.035351)
LDA: 0.785897 (0.027757)
QDA: 0.785701 (0.013667)
RDG: 0.785693 (0.027661)
ADA: 0.794968 (0.013617)
GRA: 0.809124 (0.015119)
Learning rate set to 0.026121
0:	learn: 0.6716314	total: 4.66ms	remaining: 4.66s
1:	learn: 0.6525015	total: 9.73ms	remaining: 4.86s
2:	learn: 0.6360990	total: 14.6ms	remaining: 4.86s
3:	learn: 0.6188036	total: 19.4ms	remaining: 4.83s
4:	learn: 0.6043537	total: 24ms	remaining: 4.78s
5:	learn: 0.5905753	total: 29.3ms	remaining: 4.86s
6:	learn: 0.5769956	total: 34ms	remaining: 4.82s
7:	learn: 0.5643014	total: 38.6ms	remaining: 4.78s
8:	learn: 0.5538614	total: 43.4ms	remaining: 4.78s
9:	learn: 0.5425766	total: 48.4ms	remaining: 4.79s
10:	learn: 0.5325915	total: 53.2ms	remaining: 4.79s
11:	learn: 0.5248971	total: 58.7ms	remaining: 4.83s
12:	learn: 0.5170243	total: 63.6ms	remaining: 4.83s
13:	learn: 0.5088781	total: 68.3ms

186:	learn: 0.3468169	total: 876ms	remaining: 3.81s
187:	learn: 0.3465407	total: 882ms	remaining: 3.81s
188:	learn: 0.3462598	total: 888ms	remaining: 3.81s
189:	learn: 0.3460617	total: 893ms	remaining: 3.81s
190:	learn: 0.3458636	total: 899ms	remaining: 3.81s
191:	learn: 0.3456833	total: 904ms	remaining: 3.8s
192:	learn: 0.3455794	total: 908ms	remaining: 3.8s
193:	learn: 0.3454538	total: 913ms	remaining: 3.79s
194:	learn: 0.3452271	total: 918ms	remaining: 3.79s
195:	learn: 0.3449905	total: 923ms	remaining: 3.78s
196:	learn: 0.3447041	total: 927ms	remaining: 3.78s
197:	learn: 0.3445602	total: 932ms	remaining: 3.77s
198:	learn: 0.3442950	total: 937ms	remaining: 3.77s
199:	learn: 0.3441905	total: 941ms	remaining: 3.77s
200:	learn: 0.3439457	total: 946ms	remaining: 3.76s
201:	learn: 0.3436670	total: 951ms	remaining: 3.75s
202:	learn: 0.3433682	total: 955ms	remaining: 3.75s
203:	learn: 0.3432180	total: 960ms	remaining: 3.75s
204:	learn: 0.3431309	total: 965ms	remaining: 3.74s
205:	learn: 0.

346:	learn: 0.3206209	total: 1.79s	remaining: 3.38s
347:	learn: 0.3205481	total: 1.8s	remaining: 3.37s
348:	learn: 0.3204078	total: 1.82s	remaining: 3.39s
349:	learn: 0.3203152	total: 1.82s	remaining: 3.39s
350:	learn: 0.3201290	total: 1.83s	remaining: 3.38s
351:	learn: 0.3200069	total: 1.84s	remaining: 3.38s
352:	learn: 0.3198605	total: 1.84s	remaining: 3.38s
353:	learn: 0.3197083	total: 1.85s	remaining: 3.37s
354:	learn: 0.3195817	total: 1.86s	remaining: 3.37s
355:	learn: 0.3194716	total: 1.86s	remaining: 3.37s
356:	learn: 0.3193436	total: 1.87s	remaining: 3.37s
357:	learn: 0.3192154	total: 1.89s	remaining: 3.39s
358:	learn: 0.3191331	total: 1.9s	remaining: 3.39s
359:	learn: 0.3190219	total: 1.9s	remaining: 3.38s
360:	learn: 0.3188878	total: 1.91s	remaining: 3.38s
361:	learn: 0.3188099	total: 1.91s	remaining: 3.37s
362:	learn: 0.3186424	total: 1.92s	remaining: 3.38s
363:	learn: 0.3185915	total: 1.93s	remaining: 3.37s
364:	learn: 0.3185288	total: 1.93s	remaining: 3.37s
365:	learn: 0.3

523:	learn: 0.3014321	total: 2.88s	remaining: 2.62s
524:	learn: 0.3013457	total: 2.89s	remaining: 2.61s
525:	learn: 0.3012540	total: 2.89s	remaining: 2.61s
526:	learn: 0.3011126	total: 2.9s	remaining: 2.6s
527:	learn: 0.3009978	total: 2.9s	remaining: 2.6s
528:	learn: 0.3009248	total: 2.91s	remaining: 2.59s
529:	learn: 0.3008168	total: 2.91s	remaining: 2.58s
530:	learn: 0.3007221	total: 2.92s	remaining: 2.58s
531:	learn: 0.3006067	total: 2.92s	remaining: 2.57s
532:	learn: 0.3004575	total: 2.93s	remaining: 2.56s
533:	learn: 0.3003838	total: 2.93s	remaining: 2.56s
534:	learn: 0.3003106	total: 2.94s	remaining: 2.55s
535:	learn: 0.3002201	total: 2.94s	remaining: 2.54s
536:	learn: 0.3001433	total: 2.94s	remaining: 2.54s
537:	learn: 0.3000880	total: 2.95s	remaining: 2.53s
538:	learn: 0.3000220	total: 2.95s	remaining: 2.52s
539:	learn: 0.2999614	total: 2.96s	remaining: 2.52s
540:	learn: 0.2999299	total: 2.96s	remaining: 2.51s
541:	learn: 0.2998485	total: 2.97s	remaining: 2.51s
542:	learn: 0.29

...

[04:15:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Windows10\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[04:15:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGB: 0.858229 (0.030815)
NCE: 0.707490 (0.052288)
XTC: 0.844479 (0.032468)
SGD: 0.725735 (0.080342)
PAC: 0.688650 (0.142648)
RCV: 0.785693 (0.027661)


C:\Users\Windows10\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Windows10\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Windows10\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Windows10\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Windows10\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the 

LSV: 0.745610 (0.157871)
PCP: 0.755588 (0.155843)
BNB: 0.766843 (0.049481)
DUC: 0.194196 (0.143509)
XTR: 0.833683 (0.035996)


# Leitura da Base Durante da Vacina

In [378]:
Durante= pd.read_csv('Base_Final.csv', sep= ',')
Durante= Durante.loc[Durante['CO_MUN_NOT'].isin([500270])]
Durante=Durante[(Durante['Periodo']==2.0)]
Durante= Durante.drop(columns=["CO_MUN_NOT", "Periodo"])
Durante.head()

,IDADE_ANOS,CS_SEXO_M,CS_RACA_4.0,FEBRE_1.0,DISPNEIA_1.0,SATURACAO_1.0,UTI_1.0,SUPORT_VEN_1.0,SUPORT_VEN_2.0,PCR_RESUL_2.0,TOSSE_1.0,DESC_RESP_1.0,FATOR_RISC_2,EVOLUCAO
433754,36.0,1,1,0,0,0,0,0,0,1,0,0,1,1.0
433777,0.0,1,0,0,0,0,0,0,0,1,0,0,1,1.0
433796,77.0,1,0,0,1,0,1,1,0,0,0,0,0,2.0
433826,61.0,1,0,1,1,1,0,0,1,0,1,1,0,2.0
433869,63.0,1,0,1,0,1,1,1,0,0,0,1,0,2.0


In [379]:
Durante= Durante.reset_index(drop=True)
Durante.head()

,IDADE_ANOS,CS_SEXO_M,CS_RACA_4.0,FEBRE_1.0,DISPNEIA_1.0,SATURACAO_1.0,UTI_1.0,SUPORT_VEN_1.0,SUPORT_VEN_2.0,PCR_RESUL_2.0,TOSSE_1.0,DESC_RESP_1.0,FATOR_RISC_2,EVOLUCAO
0,36.0,1,1,0,0,0,0,0,0,1,0,0,1,1.0
1,0.0,1,0,0,0,0,0,0,0,1,0,0,1,1.0
2,77.0,1,0,0,1,0,1,1,0,0,0,0,0,2.0
3,61.0,1,0,1,1,1,0,0,1,0,1,1,0,2.0
4,63.0,1,0,1,0,1,1,1,0,0,0,1,0,2.0


Verificação do Balanceamento

In [380]:
Durante["EVOLUCAO"].value_counts(normalize=True)

1.0    0.580015
2.0    0.419985
Name: EVOLUCAO, dtype: float64

Split da Base

In [381]:
X = Durante.drop(columns=['EVOLUCAO'])
y = Durante['EVOLUCAO']

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state=42)

In [382]:
Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape

((5695, 13), (2441, 13), (5695,), (2441,))

In [383]:
smote = SMOTE(sampling_strategy = 'minority', random_state = 42)
Xtrain_over, ytrain_over = smote.fit_resample(Xtrain,ytrain)

Xtest_over, ytest_over = smote.fit_resample(Xtest,ytest)
Xtrain_over.shape, ytrain_over.shape, Xtest_over.shape, ytest_over.shape

((6600, 13), (6600,), (2838, 13), (2838,))

In [384]:
# Parâmetros
num_folds = 10
scoring = 'accuracy'

# Criação dos modelos
models = []
models.append(('LR', LogisticRegression()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('BGC', BaggingClassifier()))
models.append(('RDF', RandomForestClassifier()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('QDA', QuadraticDiscriminantAnalysis()))
models.append(('RDG', RidgeClassifier()))
models.append(('ADA', AdaBoostClassifier()))
models.append(('GRA', GradientBoostingClassifier()))
models.append(('CAT', CatBoostClassifier()))
models.append(('LGB', LGBMClassifier()))
models.append(('XGB', XGBClassifier()))
#models.append(('MLP', MLPClassifier()))
models.append(('NCE', NearestCentroid()))
models.append(('XTC', ExtraTreesClassifier()))
models.append(('SGD', SGDClassifier()))
#models.append(('LSF', LabelSpreading()))
models.append(('PAC', PassiveAggressiveClassifier()))
models.append(('RCV', RidgeClassifier()))
models.append(('LSV', LinearSVC()))
models.append(('PCP', Perceptron()))
models.append(('BNB', BernoulliNB()))
models.append(('DUC', DummyClassifier()))
models.append(('XTR', ExtraTreeClassifier()))

In [385]:
np.random.seed(7) # definindo uma semente global
# Avaliação dos modelos
results = []
names = []
for name, model in models:
    kfold = KFold(n_splits=num_folds)
    cv_results = cross_val_score(model, Xtrain_over, ytrain_over, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

C:\Users\Windows10\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Windows10\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
 

LR: 0.840152 (0.012276)
KNN: 0.834242 (0.028972)
CART: 0.800000 (0.036722)
NB: 0.832576 (0.010033)
BGC: 0.827576 (0.033090)
RDF: 0.836515 (0.031429)
LDA: 0.834697 (0.025376)
QDA: 0.834545 (0.008984)
RDG: 0.834697 (0.025376)
ADA: 0.838939 (0.014502)
GRA: 0.840455 (0.010300)
Learning rate set to 0.022047
0:	learn: 0.6760968	total: 3.96ms	remaining: 3.96s
1:	learn: 0.6576411	total: 8.24ms	remaining: 4.11s
2:	learn: 0.6410282	total: 12.3ms	remaining: 4.1s
3:	learn: 0.6257906	total: 16.4ms	remaining: 4.09s
4:	learn: 0.6109434	total: 20.5ms	remaining: 4.08s
5:	learn: 0.5998011	total: 23.9ms	remaining: 3.96s
6:	learn: 0.5862115	total: 28ms	remaining: 3.98s
7:	learn: 0.5732457	total: 32.1ms	remaining: 3.98s
8:	learn: 0.5604780	total: 36.2ms	remaining: 3.98s
9:	learn: 0.5485929	total: 40.4ms	remaining: 4s
10:	learn: 0.5383920	total: 44.2ms	remaining: 3.98s
11:	learn: 0.5285879	total: 48.6ms	remaining: 4s
12:	learn: 0.5196687	total: 52.7ms	remaining: 4s
13:	learn: 0.5103196	total: 57.5ms	remaini

188:	learn: 0.3265103	total: 882ms	remaining: 3.78s
189:	learn: 0.3263359	total: 886ms	remaining: 3.78s
190:	learn: 0.3261158	total: 891ms	remaining: 3.77s
191:	learn: 0.3260022	total: 895ms	remaining: 3.77s
192:	learn: 0.3258735	total: 900ms	remaining: 3.76s
193:	learn: 0.3256711	total: 904ms	remaining: 3.76s
194:	learn: 0.3254567	total: 909ms	remaining: 3.75s
195:	learn: 0.3254526	total: 912ms	remaining: 3.74s
196:	learn: 0.3253216	total: 916ms	remaining: 3.73s
197:	learn: 0.3251668	total: 921ms	remaining: 3.73s
198:	learn: 0.3249658	total: 925ms	remaining: 3.72s
199:	learn: 0.3248415	total: 930ms	remaining: 3.72s
200:	learn: 0.3247138	total: 935ms	remaining: 3.72s
201:	learn: 0.3247100	total: 938ms	remaining: 3.7s
202:	learn: 0.3245814	total: 942ms	remaining: 3.7s
203:	learn: 0.3243602	total: 946ms	remaining: 3.69s
204:	learn: 0.3241027	total: 951ms	remaining: 3.69s
205:	learn: 0.3239629	total: 955ms	remaining: 3.68s
206:	learn: 0.3238518	total: 960ms	remaining: 3.68s
207:	learn: 0.

354:	learn: 0.3072258	total: 1.78s	remaining: 3.23s
355:	learn: 0.3070546	total: 1.78s	remaining: 3.23s
356:	learn: 0.3069247	total: 1.79s	remaining: 3.22s
357:	learn: 0.3067710	total: 1.79s	remaining: 3.22s
358:	learn: 0.3066896	total: 1.8s	remaining: 3.21s
359:	learn: 0.3065582	total: 1.81s	remaining: 3.21s
360:	learn: 0.3064393	total: 1.81s	remaining: 3.21s
361:	learn: 0.3063658	total: 1.82s	remaining: 3.2s
362:	learn: 0.3062598	total: 1.82s	remaining: 3.2s
363:	learn: 0.3061139	total: 1.83s	remaining: 3.19s
364:	learn: 0.3060257	total: 1.83s	remaining: 3.19s
365:	learn: 0.3059059	total: 1.84s	remaining: 3.19s
366:	learn: 0.3058249	total: 1.84s	remaining: 3.18s
367:	learn: 0.3057077	total: 1.85s	remaining: 3.18s
368:	learn: 0.3056171	total: 1.85s	remaining: 3.17s
369:	learn: 0.3055520	total: 1.86s	remaining: 3.17s
370:	learn: 0.3053949	total: 1.86s	remaining: 3.16s
371:	learn: 0.3053447	total: 1.87s	remaining: 3.16s
372:	learn: 0.3053015	total: 1.88s	remaining: 3.15s
373:	learn: 0.3

513:	learn: 0.2930518	total: 2.88s	remaining: 2.73s
514:	learn: 0.2929470	total: 2.89s	remaining: 2.72s
515:	learn: 0.2928777	total: 2.9s	remaining: 2.72s
516:	learn: 0.2928238	total: 2.9s	remaining: 2.71s
517:	learn: 0.2927477	total: 2.91s	remaining: 2.71s
518:	learn: 0.2926736	total: 2.91s	remaining: 2.7s
519:	learn: 0.2925723	total: 2.92s	remaining: 2.69s
520:	learn: 0.2925010	total: 2.92s	remaining: 2.69s
521:	learn: 0.2924172	total: 2.93s	remaining: 2.68s
522:	learn: 0.2923066	total: 2.93s	remaining: 2.68s
523:	learn: 0.2921685	total: 2.94s	remaining: 2.67s
524:	learn: 0.2921111	total: 2.94s	remaining: 2.66s
525:	learn: 0.2920917	total: 2.95s	remaining: 2.66s
526:	learn: 0.2919951	total: 2.95s	remaining: 2.65s
527:	learn: 0.2919047	total: 2.96s	remaining: 2.64s
528:	learn: 0.2918679	total: 2.96s	remaining: 2.64s
529:	learn: 0.2917990	total: 2.97s	remaining: 2.63s
530:	learn: 0.2917089	total: 2.97s	remaining: 2.63s
531:	learn: 0.2916511	total: 2.98s	remaining: 2.62s
532:	learn: 0.2

...

[04:16:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Windows10\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[04:16:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGB: 0.840455 (0.022134)
NCE: 0.681364 (0.020306)
XTC: 0.816818 (0.030877)
SGD: 0.813485 (0.018869)
PAC: 0.619091 (0.221354)
RCV: 0.834697 (0.025376)


C:\Users\Windows10\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Windows10\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Windows10\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Windows10\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Windows10\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the 

LSV: 0.733182 (0.120503)
PCP: 0.798182 (0.076201)
BNB: 0.821061 (0.020457)
DUC: 0.367879 (0.125717)
XTR: 0.800606 (0.035984)
